CNN RNN等等无法解决输入输出序列长度不等的问题，编码器-解码器可以解决此问题。
编码器-解码器作为高层次的设计架构，其并不与任何的具体的神经网络架构绑定。

编码器-解码器架构是一个通用的设计架构，用于解决序列到序列（sequence-to-sequence）问题，其中输入序列和输出序列的长度可能不同。
序列长度的得出，可以简单理解为利用最大似然估计，最大化输出序列基于输入序列的条件概率。

在编码器-解码器架构中，编码器是一个神经网络，它将输入序列转换为一个固定维度的向量，这个向量被称为编码器输出。然后，这个编码器输出被传递给解码器，解码器根据编码器输出和目标序列生成预测序列。以机器翻译为例：编码器依次处理源序列的每个词，最终得到语义向量c（上下文变量），解码器以句子开头为输入，根据c和句子开头生成第一个词，然后根据c和第一个词生成第二个词，以此类推，直到生成句子结尾。


In [8]:
import torch
import torch.nn as nn

#编码器
class Encoder(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers):
        super(Encoder,self).__init__()
        self.rnn=nn.RNN(input_size,hidden_size,num_layers)

    def forward(self,x,hidden):
        x,hidden=self.rnn(x,hidden) #这里的hidden相当于之前的state
        return hidden

#解码器
class Decoder(nn.Module):
    def __init__(self,output_size,hidden_size,num_layers):
        super(Decoder,self).__init__()
        self.rnn=nn.RNN(output_size,hidden_size,num_layers)
        self.linear=nn.Linear(hidden_size,output_size)
    
    def forward(self,x,hidden):
        x,state=self.rnn(x,hidden)
        x=self.linear(x)
        return x,state

#seq2seq
class seq2seq(nn.Module):
    def __init__(self,encoder,decoder):
        super().self.__init__()
        self.encoder=encoder
        self.decoder=decoder

    def forward(self,encoder_inputs,decoder_inputs):
        return self.decoder(decoder_inputs,self.encoder(encoder_inputs))
        #encoder_inputs是编码器的输入，decoder_inputs是解码器的输入

In [9]:
#假设数据集 soundmark是26个英文字母的读音，alphabet是26个英文字母
#目标是根据读音序列生成字母序列
soundmark = [
    '/eɪ/', '/biː/', '/siː/', '/diː/', '/iː/', '/ɛf/', '/dʒiː/', '/eɪtʃ/', '/aɪ/', '/dʒeɪ/',
    '/keɪ/', '/ɛl/', '/ɛm/', '/ɛn/', '/oʊ/', '/piː/', '/kjuː/', '/ɑːr/', '/ɛs/', '/tiː/',
    '/juː/', '/viː/', '/ˈdʌbəl.juː/', '/ɛks/', '/waɪ/', '/zɛd/'
]

alphabet=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']


In [10]:
import random

t=10000 #数据总条数
r=0.9 #扰动项，生成正确序列的概率，可以让数据集包含一些错误数据

seq_len=6
src_tokens,tgt_tokens=[],[] #源序列，目标序列

for i in range(t):
    src,tgt=[],[]
    for j in range(seq_len):
        ind = random.randint(0,25) #随机生成整数
        src.append(soundmark[ind])
        if random.random()<r:
            tgt.append(alphabet[ind])
        else:
            tgt.append(alphabet[random.randint(0,25)])
    src_tokens.append(src)
    tgt_tokens.append(tgt)


In [13]:
from collections import Counter

#构建匿名函数，for sublist in l :遍历外层列表l中的每一个子列表，for item in sublist：遍历每个子列表的每个元素
flatten = lambda l: [item for sublist in l for item in sublist] #展平数组

#构建词表
class Vocab:
    def __init__(self,tokens):
        self.tokens=tokens #二维列表
        self,token2index={'<bos>':0,'<eos>':1 } #两个特殊词元，分别代表开始和结束
        #将词元按照频率排序后生成列表
        self.token2index.update({
            token:index+2
            for index, (token,count) in enumerate(sorted(Counter(flatten(tokens)).items(),key=lambda x:x[1],reverse=True))
        })
        
        self.index2token={index:token for token,index in self.token2index.items()}
        
    def __getitem__(self,query): #如果query是字符串或者整数，则返回query在字典中的索引，否则返回' '<
        if isinstance(query,(str,int)): #如果query是字符串或者整数
            if isinstance(query,str):
                return self.token2index.get(query,0)  #如果query是字符串，则返回query在字典中的索引，否则返回0
            elif isinstance(query,int):
                return self.index2token.get(query,'<unk>') #如果query是整数，则返回query在字典中的索引，否则返回' '<
        elif  isinstance(query,(list,tuple)):
            return [self.__getitem__(item) for item in query]
    
    def __len__(self): #返回字典的长度
        return len(self.indef2token)   